In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

In [2]:
root = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/reports/TB回测交易记录/交易记录/"
perf = "12_18_hc_rb_debugging_imbalance.csv"
perf = pd.read_csv(root+perf, engine="python")

In [3]:
perf.head()

,公式应用,类型,商品,建仓时间,建仓价格,平仓时间,平仓价格,数量,交易成本,净利,累计净利,收益率,累计收益率
0,L1_S2_commodities_only_1_pair,空头,hc000,2014-10-30,2852,2015-3-19,2514,262,10620.59,874939.41,874939.41,30678.10%,30678.10%
1,L1_S2_commodities_only_1_pair,多头,rb000,2014-10-30,2611,2015-3-19,2459,287,11625.51,-447865.51,427073.90,-17153.03%,13525.07%
2,L1_S2_commodities_only_1_pair,空头,rb000,2015-3-19,2459,2015-12-23,1723,305,12327.55,2232472.45,2659546.35,90787.82%,104312.89%
3,L1_S2_commodities_only_1_pair,多头,hc000,2015-3-19,2514,2015-12-23,1854,298,12050.17,-1978850.17,680696.18,-78713.21%,25599.68%
4,L1_S2_commodities_only_1_pair,空头,hc000,2015-12-23,1854,2017-3-1,3580,404,16379.53,-6989419.53,-6308723.35,-376991.34%,-351391.67%


In [16]:
# del perf['Unnamed: 0']

In [4]:
perf.columns = ['strat', 'position_direction', 'asset', 'entry_date', 'entry_price', 'exit_date', 'exit_price', 'qty', 'trade_costs', 'net_gains', 'cum_gains',
       'returns', 'cum_returns']

In [5]:
perf

,strat,position_direction,asset,entry_date,entry_price,exit_date,exit_price,qty,trade_costs,net_gains,cum_gains,returns,cum_returns
0,L1_S2_commodities_only_1_pair,空头,hc000,2014-10-30,2852,2015-3-19,2514,262,10620.59,874939.41,874939.41,30678.10%,30678.10%
1,L1_S2_commodities_only_1_pair,多头,rb000,2014-10-30,2611,2015-3-19,2459,287,11625.51,-447865.51,427073.90,-17153.03%,13525.07%
2,L1_S2_commodities_only_1_pair,空头,rb000,2015-3-19,2459,2015-12-23,1723,305,12327.55,2232472.45,2659546.35,90787.82%,104312.89%
3,L1_S2_commodities_only_1_pair,多头,hc000,2015-3-19,2514,2015-12-23,1854,298,12050.17,-1978850.17,680696.18,-78713.21%,25599.68%
4,L1_S2_commodities_only_1_pair,空头,hc000,2015-12-23,1854,2017-3-1,3580,404,16379.53,-6989419.53,-6308723.35,-376991.34%,-351391.67%
5,L1_S2_commodities_only_1_pair,多头,rb000,2015-12-23,1723,2017-3-1,3509,435,17627.59,7751472.41,1442749.06,449882.32%,98490.66%
6,L1_S2_commodities_only_1_pair,空头,rb000,2017-3-1,3509,2018-12-14,3468,213,8668.61,78661.39,1521410.45,2241.70%,100732.36%


# Create a function that transforms the raw data table

In [6]:
def transform(perf):
    new_df = pd.DataFrame([])
    new_df["date"] = perf['entry_date']
    new_df['strat'] = perf['strat']
    new_df['asset'] = perf['asset']
    new_df['entry_date'] = perf['entry_date']
    new_df['entry_price'] = perf['entry_price']
    new_df['qty'] = perf['qty']
    new_df['pos_dirctn'] = perf['position_direction']
    
    all_dates = list(perf['entry_date'].append(perf['exit_date']))
    all_strats = list(perf['strat'].append(perf['strat']))
    all_assets = list(perf['asset'].append(perf['asset']))
    all_qty = list(perf['qty'].append(perf['qty']))
    all_pos_dirctn = list(perf['position_direction'].append(perf['position_direction']))
    all_exit_price = perf['exit_price']
    
    new_df = pd.DataFrame([])
    new_df['all_dates'] = all_dates
    new_df['all_strats'] = all_strats
    new_df['assets'] = all_assets
    new_df['pos_dirctn'] = all_pos_dirctn
    new_df['qty'] = all_qty
    
    new_df['entry_price'] = perf['entry_price']
    new_df['exit_price'] = ''
    new_df['exit_price'][len(perf):] = all_exit_price
    
    new_df['all_dates'] = pd.to_datetime(new_df['all_dates'])
    new_df.sort_values('all_dates', inplace=True)
    new_df.fillna("", inplace=True)
    new_df['all_dates'] = new_df['all_dates'].astype(str)
    
    return new_df

In [7]:
new_df = transform(perf)

C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
new_df[['all_dates', 'assets', 'pos_dirctn', 'qty', 'entry_price', 'exit_price']].to_csv(root+"12_18_debuggging_imbalance_check_positions.csv", index = False)

In [11]:
# new_df[(new_df['all_dates']>'2018-11-28')&(new_df['all_dates']<'2018-12-16')]

# See fixed version  

In [13]:
perf_fixed = "12_18_hc_rb_debugging_imbalance_fixed.csv"

In [14]:
perf_fixed = pd.read_csv(root+perf_fixed, engine="python")

In [16]:
perf_fixed.columns = ['strat', 'position_direction', 'asset', 'entry_date', 'entry_price', 'exit_date', 'exit_price', 'qty', 'trade_costs', 'net_gains', 'cum_gains',
       'returns', 'cum_returns']

In [18]:
new_df_fixed = transform(perf_fixed)

C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
new_df_fixed

,all_dates,all_strats,assets,pos_dirctn,qty,entry_price,exit_price
0,2014-10-30,L1_S2_commodities_1_pair_tryfix,hc000,空头,262,2852,
1,2014-10-30,L1_S2_commodities_1_pair_tryfix,rb000,多头,287,2611,
2,2015-03-19,L1_S2_commodities_1_pair_tryfix,rb000,空头,305,2459,
3,2015-03-19,L1_S2_commodities_1_pair_tryfix,hc000,多头,298,2514,
17,2015-03-19,L1_S2_commodities_1_pair_tryfix,rb000,多头,287,,2459
16,2015-03-19,L1_S2_commodities_1_pair_tryfix,hc000,空头,262,,2514
4,2015-12-23,L1_S2_commodities_1_pair_tryfix,hc000,空头,404,1854,
5,2015-12-23,L1_S2_commodities_1_pair_tryfix,rb000,多头,435,1723,
19,2015-12-23,L1_S2_commodities_1_pair_tryfix,hc000,多头,298,,1854
18,2015-12-23,L1_S2_commodities_1_pair_tryfix,rb000,空头,305,,1723
